<a href="https://colab.research.google.com/github/ashikita/pdf_adjustment/blob/main/pdf_encrypt_copy_restrict.py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

# -*- coding: utf-8 -*-
"""
PDFの本文コピーを制限する

"""

# === 設定ブロック ===
USER_PASSWORD = "your_user_password"   # 閲覧時に必要なパスワード
OWNER_PASSWORD = "your_owner_password" # すべての権限を持つパスワード（未設定なら USER_PASSWORD と同じにする）
ALGORITHM_PREFS = ("AES-256", "AES-128", "RC4-128")  # 利用可能なものがあれば順に使う
ALLOW_ACCESSIBILITY = True  # スクリーンリーダー用のテキスト抽出を許可する場合は True

# ---- 依存パッケージ（必要なら pip で自動インストール）----
import sys
import subprocess

def _pip_install(pkg):
    # 失敗時にも詳細が見えるように stdout/stderr を親プロセスに出す
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", pkg])

# まず pypdf を優先
try:
    import pypdf  # 新名称（推奨）
    from pypdf import PdfReader, PdfWriter
    try:
        from pypdf import Permissions  # pypdf>=3 目安
    except Exception:
        Permissions = None
    _LIB = "pypdf"
except ImportError:
    # 未導入ならインストールを試行
    try:
        _pip_install("pypdf")
        import pypdf
        from pypdf import PdfReader, PdfWriter
        try:
            from pypdf import Permissions
        except Exception:
            Permissions = None
        _LIB = "pypdf"
    except Exception:
        # pypdf が使えない環境では PyPDF2 にフォールバック
        try:
            import PyPDF2
            from PyPDF2 import PdfReader, PdfWriter
            try:
                from PyPDF2.constants import Permissions  # PyPDF2>=2.12 目安
            except Exception:
                Permissions = None
            _LIB = "PyPDF2"
        except ImportError:
            # PyPDF2 も無ければインストール
            _pip_install("PyPDF2")
            import PyPDF2
            from PyPDF2 import PdfReader, PdfWriter
            try:
                from PyPDF2.constants import Permissions
            except Exception:
                Permissions = None
            _LIB = "PyPDF2"

print(f"[INFO] Using {_LIB}")

# === ここから実装 ===
import os
import time
import glob

# 許可する権限セット（印刷のみ許可、本文コピーは禁止）
allowed_permissions = None
if Permissions is not None:
    # 印刷のみ許可（コピーは指定しない＝禁止）
    # アクセシビリティ（スクリーンリーダー）許可は任意
    allowed_permissions = set()
    if hasattr(Permissions, "PRINTING"):
        allowed_permissions.add(getattr(Permissions, "PRINTING"))
    if ALLOW_ACCESSIBILITY and hasattr(Permissions, "ACCESSIBILITY"):
        allowed_permissions.add(getattr(Permissions, "ACCESSIBILITY"))
    if not allowed_permissions:
        # 何も見つからない場合は None（ライブラリ側デフォルトに委ねる）
        allowed_permissions = None

def encrypt_with_best_algorithm(writer, user_pwd, owner_pwd, perms):
    """
    ライブラリやバージョン差異に合わせて encrypt 呼び出しを試行。
    ・permissions を明示（対応していればコピー不可・印刷可）
    ・アルゴリズムは優先リストで利用可のものを採用
    """
    if not owner_pwd:
        owner_pwd = user_pwd

    # permissions と algorithm の両方を受け付ける実装（pypdf 新しめ）
    for algo in ALGORITHM_PREFS:
        try:
            if perms is not None:
                writer.encrypt(
                    user_password=user_pwd,
                    owner_password=owner_pwd,
                    permissions=perms,
                    algorithm=algo,
                )
            else:
                writer.encrypt(
                    user_password=user_pwd,
                    owner_password=owner_pwd,
                    algorithm=algo,
                )
            return
        except TypeError:
            # 引数名違い・未対応
            pass
        except Exception:
            # 未サポートのアルゴリズムなど
            pass

    # algorithm 指定なし
    try:
        if perms is not None:
            writer.encrypt(
                user_password=user_pwd,
                owner_password=owner_pwd,
                permissions=perms,
            )
        else:
            writer.encrypt(
                user_password=user_pwd,
                owner_password=owner_pwd,
            )
        return
    except TypeError:
        pass

    # password として受け取る古い API
    try:
        if perms is not None:
            writer.encrypt(
                password=user_pwd,
                owner_password=owner_pwd,
                permissions=perms,
            )
        else:
            writer.encrypt(
                password=user_pwd,
                owner_password=owner_pwd,
            )
        return
    except TypeError:
        pass

    # さらに古い PyPDF2 互換（permissions なし・128bit 固定など）
    try:
        writer.encrypt(user_pwd, owner_pwd, use_128bit=True)
        return
    except TypeError:
        writer.encrypt(user_pwd, owner_pwd)

def process_pdf(in_path, out_path):
    # 読み込み（暗号化済み PDF への対処）
    reader = PdfReader(in_path)
    if getattr(reader, "is_encrypted", False):
        # 空パスワードで開けるタイプへの対処（失敗時は無視）
        try:
            if hasattr(reader, "decrypt"):
                reader.decrypt("")
        except Exception:
            pass

    writer = PdfWriter()

    # ページコピー
    for page in reader.pages:
        writer.add_page(page)

    # メタデータの継承（失敗しても続行）
    try:
        meta = getattr(reader, "metadata", None)
        if meta:
            writer.add_metadata({k: v for k, v in meta.items() if v is not None})
    except Exception:
        pass

    # 暗号化設定（コピー不可・印刷可）
    encrypt_with_best_algorithm(
        writer,
        USER_PASSWORD,
        OWNER_PASSWORD or USER_PASSWORD,
        allowed_permissions,
    )

    # 書き出し
    with open(out_path, "wb") as f:
        writer.write(f)

def main():
    os.makedirs("output", exist_ok=True)

    # カレント直下の PDF（output ディレクトリは対象外）
    candidates = []
    for pattern in ("*.pdf", "*.PDF", "*.Pdf", "*.pDf", "*.pdF"):
        candidates.extend(glob.glob(pattern))
    candidates = [p for p in sorted(set(candidates)) if not p.startswith("output"+os.sep)]

    if not candidates:
        print("INFO: 対象となる PDF が見つかりませんでした。")
        return

    for pdf_path in candidates:
        start = time.perf_counter()
        status = "SUCCESS"
        err_msg = ""
        try:
            out_path = os.path.join("output", os.path.basename(pdf_path))
            process_pdf(pdf_path, out_path)
        except Exception as e:
            status = "ERROR"
            err_msg = str(e)
        elapsed = time.perf_counter() - start

        # 要求仕様：ファイル名、成功・エラーのステータス、かかった時間
        if status == "SUCCESS":
            print(f"{os.path.basename(pdf_path)},SUCCESS,{elapsed:.3f}s")
        else:
            print(f"{os.path.basename(pdf_path)},ERROR,{elapsed:.3f}s,{err_msg}")

if __name__ == "__main__":
    main()


[INFO] Using pypdf
9203_c01.pdf,SUCCESS,0.041s
9203_c02.pdf,SUCCESS,0.024s
9203_colophon.pdf,SUCCESS,0.030s
9203_commemoration.pdf,SUCCESS,0.028s
9203_contents.pdf,SUCCESS,0.028s
9203_coversheet.pdf,SUCCESS,0.020s
9203_list.pdf,SUCCESS,0.049s
9203_p001.pdf,SUCCESS,0.073s
9203_p021.pdf,SUCCESS,0.075s
9203_p043.pdf,SUCCESS,0.098s
9203_p075.pdf,SUCCESS,0.075s
9203_p095.pdf,SUCCESS,0.090s
9203_p121.pdf,SUCCESS,0.099s
9203_p151.pdf,SUCCESS,0.245s
9203_p183.pdf,SUCCESS,0.128s
9203_rule.pdf,SUCCESS,0.025s
